In [1]:
!pip install pykeops[full]


     |████████████████████████████████| 358kB 5.3MB/s 
     |████████████████████████████████| 6.5MB 7.4MB/s 
     |████████████████████████████████| 2.8MB 29.8MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 286kB 39.9MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for pykeops: filename=pykeops-1.4.2-cp36-none-any.whl size=485954 sha256=465f5459eabe4055dc3b11a105c62b8d5a05f4b4041529190d22c9e304489e69
  Stored in directory: /root/.cache/pip/wheels/9b/ec/76/94d034cbec746ed30b447710eaa0c64ba25cb0a85a7f0f649d
  Created wheel for sphinx-gallery: filename=sphinx_gallery-0.8.2-cp36-none-any.whl size=124839 sha256=87031fc7e9003b6260ba387ec4ac0d69f1f3040d75df34116d792488454ffd51
  Stored in directory: /root/.cache/pip/wheels/9b/5c/9b/f13edd33356677da0a1cd5a3998194e9fd46f2c3e992d8bde0
  Created wheel for gpytorch: filename=gpytorch-1.3.1-py2.py3-none-any.whl size=474648 sha256=a6c90ce7098e7164e5b3b3f8a1f426d9eb74e6c9

# Nystrom implementation

In [ ]:
from sklearn import datasets, svm
import time
import numpy as np
import torch
from matplotlib import pyplot as plt
from pykeops.torch import LazyTensor
from scipy.linalg import svd

In [ ]:
def pairwise_kernels_keops(x,y=None, sigma=0.1):
  """A kernel matrix K such that K_{i, j} is the kernel between the ith and jth 
  vectors of the given matrix X"""
  #TODO: check pairwise arrays etc for Value Error
  if y==None:
    y=x
  x_i = LazyTensor(x[:, None, :])  # (M, 1, 1)
  y_j = LazyTensor(y[None, :, :])  # (1, N, 1)
  print(123)
  D_ij = ((x_i - y_j) ** 2).sum(-1)  # (M, N) symbolic matrix of squared distances
  print(type(D_ij))
  return (-D_ij / (2 * sigma ** 2)).exp()  # (M, N) symbolic Gaussian kernel matrix
  


def pairwise_kernels(x, y=None, sigma =0.1):
  if y==None:
    y=x
  x_i = x[:, None, :]  # (M, 1, 1)
  y_j = y[None, :, :] # (1, N, 1)
  D_ij = ((x_i-y_j) ** 2).sum(-1)  # (M, N) symbolic matrix of squared distances
  print(type(D_ij))
  return (-D_ij / (2 * sigma ** 2)).exp()  # (M, N) symbolic Gaussian kernel matrix
    



In [ ]:
class Nystrom:
    '''Class to implement Nystrom'''

  def __init__(self, n_components=100):
    self.n_components = n_components
    #self.normalization_=None

  def fit(self, X, y):
    #number of samples. must be greater than n_components
    n_samples = X.shape[0]

    #define basis
    n_components = min(n_samples, self.n_components) #should be = n_samples
    inds = torch.randperm(n_samples) #rnd.permutation(n_samples)
    basis_inds = inds[:n_components]
    basis = X[basis_inds]
    # get rbf kernel matrix
    basis_kernel = pairwise_kernels(basis, y)

    # get svd
    U, S, V = torch.svd(basis_kernel) 
    u = U * 1. / torch.sqrt(S)
    self.normalization_ = torch.dot(u[:,0], V[:,0]) #????
    self.components_ = basis
    self.component_indices_ = inds
   
    return basis_kernel


